In [ ]:
import torch

from diffusers import DiffusionPipeline
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file


import sys
sys.path.append('.')
from utils.load_util import load_pipe
from utils.lora import LoRANetwork, DEFAULT_TARGET_REPLACE, UNET_TARGET_REPLACE_MODULE_CONV


weights_dtype = torch.bfloat16
device = 'cuda:0'



# Transfer

In [ ]:
import torch

from diffusers import DiffusionPipeline
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file


import sys
sys.path.append('.')
from utils.load_util import load_pipe
from utils.lora import LoRANetwork, DEFAULT_TARGET_REPLACE, UNET_TARGET_REPLACE_MODULE_CONV


weights_dtype = torch.bfloat16
device = 'cuda:0'



pipe = load_pipe(distillation_type='lcm', weights_dtype=weights_dtype, device=device)


pipe.to(device, weights_dtype)
print('loaded model')

unet=pipe.unet
## Change these parameters based on how you trained your sliderspace sliders
train_method = 'noxattn'
rank = 4
alpha =1 
print(rank, alpha, train_method)
networks = {}
modules = DEFAULT_TARGET_REPLACE
modules += UNET_TARGET_REPLACE_MODULE_CONV

network = LoRANetwork(
    unet,
    rank=int(rank),
    multiplier=1.0,
    alpha=int(alpha),
    train_method=train_method,
    fast_init=True,
).to(device, dtype=weights_dtype)

In [ ]:
import random
import matplotlib.pyplot as plt
slider_path = 'eyesize.pt'
network.load_state_dict(torch.load(slider_path))


for _ in range(20):
    seed = random.randint(0, 2**15)
    
    all_images = []
    for scale in [2, 0, -2]:
        network.set_lora_slider(scale=scale)
        with network:
            pass
    
        generator = torch.Generator().manual_seed(seed)
        
        prompt = 'image of a black female person, realistic, 8k'
        with network:
            image = pipe(prompt=prompt,  num_inference_steps=4, guidance_scale=0, generator=generator)[0]
    
        all_images.append(image)
    
    # Create 5x5 grid with no whitespace
    fig, axes = plt.subplots(1, 3, figsize=(9, 3), dpi=200)
    plt.subplots_adjust(wspace=0, hspace=0)  # Remove spacing between subplots
    fig.patch.set_visible(False)  # Hide the figure's background
    for i, ax in enumerate(axes.flat):
        ax.imshow(all_images[i])
        ax.axis('off')  # Remove axes
        ax.set_xticklabels([])  # Remove tick labels
        ax.set_yticklabels([])
        ax.tick_params(left=False, bottom=False)  # Remove ticks
    
    # Remove any remaining borders/margins
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.show()